In [2]:
print('test')

test


In [3]:
# Set the base directory for the project
base_dir = "/opt/ml/processing"

In [4]:
# Create the necessary directories for the project 
%mkdir -p {base_dir}/train/
%mkdir -p {base_dir}/validation/
%mkdir -p {base_dir}/test/
%mkdir -p {base_dir}/modeldata/

In [5]:
# Install the LightGBM library
# LightGBM is a gradient boosting framework that uses tree-based learning algorithms
%pip install lightgbm

  Using cached lightgbm-4.5.0-py3-none-manylinux_2_28_x86_64.whl.metadata (17 kB)
Using cached lightgbm-4.5.0-py3-none-manylinux_2_28_x86_64.whl (3.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
# Import the necessary libraries and modules for the project
import datetime;

import argparse
import logging
import pathlib

import boto3
import numpy as np
import pandas as pd
import numpy as np

In [32]:
#Import commonly used machine learning libraries for data preprocessing and model selection.
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [8]:
# Get the current date and time and format it as a string
datenow = datetime.datetime.now().strftime("%Y-%m-%d-%H%M")
print(datenow)

2024-10-28-2147


In [10]:
# Set main variables for the EDA
base_dir = "/opt/ml/processing"
default_bucket = 'z-tmp-tfc'
input_data = f"s3://{default_bucket}/data/heart_disease_uci.csv"  # Change this to point to the s3 location of your raw input data.
bucket = input_data.split("/")[2]
key = "/".join(input_data.split("/")[3:])
fn = f"{base_dir}/data/heart_disease_uci.csv"
#print(input_data)

In [11]:
# Creates the data directory if it doesn't exist, and downloads data from S3 to the local machine.
pathlib.Path(f"{base_dir}/data").mkdir(parents=True, exist_ok=True)
s3 = boto3.resource("s3")
s3.Bucket(bucket).download_file(key, fn)

In [9]:
# Begin The EDA 

In [12]:
# read in csv
df = pd.read_csv(fn)

In [ ]:
# Displays sample of the DataFrame.

In [13]:
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [15]:
# Removes the 'id' column from the DataFrame
df.drop('id', axis=1, inplace=True)    

In [16]:
# replace zero values with NaN in cholestrol column
df['chol'] = df['chol'].replace(0, np.NaN)

In [18]:
# encode the data using label encoding
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['sex', 'dataset', 'cp', 'thal', 'slope', 'exang', 'restecg', 'fbs']
num_cols = [col for col in df.select_dtypes(exclude='O')]

label_encoders = {}
data = df.copy()
index = {}

for colm in columns_to_encode:
    nan_ixs = np.where(data[colm].isna())[0]
    index[colm] = nan_ixs

In [21]:
#Identify numeric and categorical columns with missing values in the DataFrame.
cols_with_nans = [x for x in df if df[x].isnull().sum() > 0]
num = [col for col in df[cols_with_nans].select_dtypes(exclude='O')]
cat = [col for col in df[cols_with_nans].select_dtypes(include='O')]
num

['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']

In [27]:
#Numeric columns
numeri_cols = ['trestbps', 'chol', 'thalch', 'oldpeak']

In [22]:
# Apply Encode categorical columns, store label encoders, and replace encoded values with NaNs.
# The entire data set must be represented by numbers
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

    label_encoders[col] = le

    for col, idxs in index.items():
        df.loc[idxs, col] = np.nan

In [23]:
df.head()

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,1,0,3,145.0,233.0,1.0,0.0,150.0,0.0,2.3,0.0,0.0,0.0,0
1,67,1,0,0,160.0,286.0,0.0,0.0,108.0,1.0,1.5,1.0,3.0,1.0,2
2,67,1,0,0,120.0,229.0,0.0,0.0,129.0,1.0,2.6,1.0,2.0,2.0,1
3,37,1,0,2,130.0,250.0,0.0,1.0,187.0,0.0,3.5,0.0,0.0,1.0,0
4,41,0,0,1,130.0,204.0,0.0,0.0,172.0,0.0,1.4,2.0,0.0,1.0,0


In [24]:
#List of categorical columns in the DataFrame.
categ_cols = ['fbs', 'restecg', 'exang', 'slope', 'thal', 'ca']

In [31]:
#Impute missing values in numeric columns using a LightGBM regression model.
#from sklearn.preprocessing import MinMaxScaler
for col in numeri_cols:

    df_with_missing = df[df[col].isna()]
    # dropna removes all rows with missing values
    df_without_missing = df[df[col].notna()]

    # split the data into X and y and we will only take the columns with no missing values
    X = df_without_missing.drop([col], axis=1)
    y = df_without_missing[col]

    # split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    # Random Forest Imputation
    rf_model = LGBMRegressor(
        objective = "regression",
        metric = "rmse",
        n_estimators =  1000,
        bagging_freq = 1,subsample = 0.413103572972995, 
                             colsample_bytree = 0.5816717344110182,
                             min_data_in_leaf = 20,
                             learning_rate = 0.004730072022055302,
                             num_leaves = 364, verbose = -1 ,random_state=42)
    rf_model.fit(X_train, y_train)

    # evaluate the model
    y_preds = rf_model.predict(X_test)
    print("Missing Values", col, ":", str(round((df[col].isnull().sum() / len(df)) * 100, 2))+"%")

    #Apply the trained model to predict missing values
    y_pred = np.round(rf_model.predict(df_with_missing.drop([col], axis=1)))
    df_with_missing[col] = y_pred

    #concatenate the imputed DataFrame.
    df = pd.concat([df_with_missing, df_without_missing], axis=0)


Missing Values trestbps : 6.41%


/tmp/ipykernel_1461/3011436372.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred


Missing Values chol : 21.96%


/tmp/ipykernel_1461/3011436372.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred


Missing Values thalch : 5.98%


/tmp/ipykernel_1461/3011436372.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred


Missing Values oldpeak : 6.74%


/tmp/ipykernel_1461/3011436372.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred


In [33]:
#Impute missing values in categorical columns using a LightGBM model and evaluate the accuracy.
for col in categ_cols: 
    df_with_missing = df[df[col].isna()]
    # dropna removes all rows with missing values
    # df_without_missing = df.dropna()
    df_without_missing = df[df[col].notna()]

    # split the data into X and y and we will only take the columns with no missing values
    X = df_without_missing.drop([col], axis=1)
    y = df_without_missing[col]

    # split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    # Random Forest Imputation
    rf_model = LGBMClassifier(verbose = -1,learning_rate = 0.023021779601797816, num_leaves = 149, subsample = 0.6929884706542179, colsample_bytree = 0.8635308367372507, min_data_in_leaf = 47, random_state=42)
    rf_model.fit(X_train, y_train)

    # evaluate the model
    y_preds = rf_model.predict(X_test)
    y_pred = rf_model.predict(df_with_missing.drop([col], axis=1))
    acc_score = accuracy_score(y_test, y_preds)

    # print result
    print("The feature '"+ col+ "' has been imputed with", round((acc_score * 100), 2), "accuracy\n")
    df_with_missing[col] = y_pred
    
    # apply the predicted values to the missing data.
    df = pd.concat([df_with_missing, df_without_missing], axis=0)

The feature 'fbs' has been imputed with 82.53 accuracy

The feature 'restecg' has been imputed with 64.67 accuracy

The feature 'exang' has been imputed with 80.92 accuracy



/tmp/ipykernel_1461/240859716.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred
/tmp/ipykernel_1461/240859716.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred
/tmp/ipykernel_1461/240859716.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

The feature 'slope' has been imputed with 62.6 accuracy

The feature 'thal' has been imputed with 72.41 accuracy

The feature 'ca' has been imputed with 50.0 accuracy



/tmp/ipykernel_1461/240859716.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred
/tmp/ipykernel_1461/240859716.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_missing[col] = y_pred
/tmp/ipykernel_1461/240859716.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [34]:
#Convert the encoded categorical columns to integer data type.
for i in columns_to_encode:
    df[i] = df[i].astype(int)

In [36]:
df.dtypes

age           int64
sex           int64
dataset       int64
cp            int64
trestbps    float64
chol        float64
fbs           int64
restecg       int64
thalch      float64
exang         int64
oldpeak     float64
slope         int64
ca          float64
thal          int64
num           int64
dtype: object

In [37]:
df.head()

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
733,63,1,3,2,136.0,253.0,0,0,128.0,0,0.0,1,1.0,1,1
743,74,1,3,2,136.0,246.0,0,1,129.0,0,0.0,1,0.0,1,0
746,51,1,3,0,135.0,258.0,1,2,131.0,1,1.0,1,1.0,2,2
750,54,1,3,0,132.0,247.0,0,2,121.0,1,1.0,1,1.0,2,3
771,63,0,3,1,136.0,255.0,0,1,133.0,0,0.0,2,0.0,1,0


In [38]:
#Reverse the label encoding and restore the original categorical values in the DataFrame.
for col in columns_to_encode:
    # Retrieve the corresponding LabelEncoder for the column
    le = label_encoders[col]

    # Inverse transform the data
    df[col] = le.inverse_transform(df[col]).astype('O')

In [39]:
df.head()

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
733,63,Male,VA Long Beach,non-anginal,136.0,253.0,False,lv hypertrophy,128.0,False,0.0,flat,1.0,normal,1
743,74,Male,VA Long Beach,non-anginal,136.0,246.0,False,normal,129.0,False,0.0,flat,0.0,normal,0
746,51,Male,VA Long Beach,asymptomatic,135.0,258.0,True,st-t abnormality,131.0,True,1.0,flat,1.0,reversable defect,2
750,54,Male,VA Long Beach,asymptomatic,132.0,247.0,False,st-t abnormality,121.0,True,1.0,flat,1.0,reversable defect,3
771,63,Female,VA Long Beach,atypical angina,136.0,255.0,False,normal,133.0,False,0.0,upsloping,0.0,normal,0


In [40]:
# it is not physiologically possible for resting blood pressure in mm Hg to be zero.
df = df[df['trestbps'] != 0]

In [41]:
# Separate numerical and categorical features
num_cols = [col for col in df.select_dtypes(exclude='O')]
cat_cols = [col for col in df.select_dtypes(include='O')]
num_cols

['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca', 'num']

In [42]:
#Split the DataFrame into numeric and categorical columns for further processing.
n = df[['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca', 'num']]
num_cols = n
cat_cols = pd.DataFrame (df, columns= cat_cols)

In [43]:
#Drop duplicate data
df.drop_duplicates(inplace=True)   

In [44]:
#Bin the numeric 'num' column into categories based on predefined thresholds.
bins=[0,1,2,3,4]
labels=['No-Heart-Disease', 'Mild-Heart-Disease', 'Moderate-Heart-Disease', 'Severe-Heart-Disease']
df['num_bins']=pd.cut(df['num'], bins=bins, labels=labels, include_lowest=True)

In [45]:
#Encode categorical columns using LabelEncoder and store the encoders for later reference.
label_encoders ={}
for col in df[['sex', 'dataset', 'cp', 'thal', 'slope', 'exang', 'restecg', 'fbs','num_bins' ]]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [47]:
#Rearrange the columns in the DataFrame, moving the last column to the front.
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [48]:
#Drop num and keep the new bin target.
df.drop('num', axis=1, inplace=True)

In [49]:
df.head()

,num_bins,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
733,2,63,1,3,2,136.0,253.0,0,0,128.0,0,0.0,1,1.0,1
743,2,74,1,3,2,136.0,246.0,0,1,129.0,0,0.0,1,0.0,1
746,0,51,1,3,0,135.0,258.0,1,2,131.0,1,1.0,1,1.0,2
750,1,54,1,3,0,132.0,247.0,0,2,121.0,1,1.0,1,1.0,2
771,2,63,0,3,1,136.0,255.0,0,1,133.0,0,0.0,2,0.0,1


In [50]:
#save the datasets
model_data = df

pd.DataFrame(model_data).to_csv(
    f"{base_dir}/modeldata/fulldataset.csv", header=True, index=False
)

In [51]:
# Split the data
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)

pd.DataFrame(train_data).to_csv(
    f"{base_dir}/train/train.csv", header=False, index=False
)
pd.DataFrame(validation_data).to_csv(
    f"{base_dir}/validation/validation.csv", header=False, index=False
)
pd.DataFrame(test_data).to_csv(
    f"{base_dir}/test/test.csv", header=False, index=False
)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [52]:
#Upload Datasets to S3
s3 = boto3.resource('s3')
s3.Bucket(bucket).upload_file(f"{base_dir}/train/train.csv", "data/train/train.csv")
s3.Bucket(bucket).upload_file(f"{base_dir}/validation/validation.csv", "data/validation/validation.csv")
s3.Bucket(bucket).upload_file(f"{base_dir}/test/test.csv", "data/test/test.csv")
s3.Bucket(bucket).upload_file(f"{base_dir}/modeldata/fulldataset.csv", "data/modeldata/fulldataset.csv")

In [ ]:
#### TRAINING
####

In [54]:
import sagemaker
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [58]:
# Set up init variables
region = 'us-east-1'
prefix = 'data'
# output path where the trained model will be saved
output_path = 's3://{}/{}/{}/output'.format(bucket, prefix, 'tfc-heart-xgb')
# define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput("s3://{}/{}/{}/".format(bucket, prefix, 'train'), content_type=content_type)
validation_input = TrainingInput("s3://{}/{}/{}/".format(bucket, prefix, 'validation'), content_type=content_type)

In [56]:
# initialize hyperparameters
hyperparameters = {
        "objective":"multi:softmax",
        "num_class":"5",
        "max_depth":"4",
        "learning_rate":"0.2",
        "gamma":"0.1",
        "subsample":"1.0",
        "reg_alpha":"0",
        "reg_lambda":"1",
        "early_stopping_rounds":"10",
        "num_round":"100"}

In [59]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          base_job_name='tfc-job--xgb-heart-' + datenow,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

# execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker:Creating training-job with name: tfc-job--xgb-heart-2024-10-28-2147-2024-10-28-22-28-23-443


2024-10-28 22:28:23 Starting - Starting the training job...
2024-10-28 22:28:40 Starting - Preparing the instances for training...
2024-10-28 22:29:11 Downloading - Downloading input data...
2024-10-28 22:29:31 Downloading - Downloading the training image...
2024-10-28 22:30:08 Training - Training image download completed. Training in progress...[2024-10-28 22:30:29.197 ip-10-0-130-138.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-28 22:30:29.224 ip-10-0-130-138.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-28:22:30:29:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-28:22:30:29:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2024-10-28:22:30:29:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-28:22:30:29:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-10-28:22:30:29:INFO] Determined 0 GPU(s) available on the instan

In [ ]:
#### TRAINING - not working
####

In [60]:
# initialize hyperparameters
hyperparameters = {
        "predictor_type":"multiclass_classifier",
        "num_classes":"5",
        "feature_dim":"14",
        "mini_batch_size":"200"}

In [61]:
# output path where the trained model will be saved
output_path = 's3://{}/{}/{}/output'.format(bucket, prefix, 'tfc-heart-linear')

In [62]:
# this line automatically looks for the linear-learner image URI
linear_container = sagemaker.image_uris.retrieve(region=region, framework="linear-learner")

# construct a SageMaker estimator that calls the linear-learner 
estimator = sagemaker.estimator.Estimator(image_uri=linear_container, 
                                          hyperparameters=hyperparameters,
                                          base_job_name='tfc-job-linear-heart-',
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)

# define the content type
content_type = "text/csv"

# execute the linear-learner training job
estimator.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: tfc-job-linear-heart--2024-10-28-22-32-14-629


2024-10-28 22:32:14 Starting - Starting the training job...
2024-10-28 22:32:29 Starting - Preparing the instances for training...
2024-10-28 22:33:12 Downloading - Downloading the training image.........
2024-10-28 22:34:23 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[10/28/2024 22:34:45 INFO 140031712307008] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insensit

UnexpectedStatusException: Error for Training job tfc-job-linear-heart--2024-10-28-22-32-14-629: Failed. Reason: ClientError: No iterator has been registered for ContentType ('csv', '1.0'), exit code: 2